<a href="https://colab.research.google.com/github/ksur/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


%load_ext tensorboard

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, 
            y_train,
            batch_size=params_fit.get('batch_size', 128),
            epochs=params_fit.get('epochs', 5),
            verboses=params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', (X_train, y_train)),
            callbacks=[tensorboard_callback]
            )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same"),
                    MaxPool2D(),
                    Dropout(0.3),
                    
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
                    MaxPool2D(),
                    Dropout(0.3),
                    
                    Flatten(),
                    
                    Dense(1024, activation="relu"),
                    Dropout(0.3),

                    Dense(1024, activation="relu"),
                    Dropout(0.3),

                    Dense(num_classes, activation='softmax'),
                    ])

In [0]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 47ms/step - loss: 2.7164 - accuracy: 0.2707 - val_loss: 1.1377 - val_accuracy: 0.6464
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 1.0536 - accuracy: 0.6625 - val_loss: 0.3929 - val_accuracy: 0.8863
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.5542 - accuracy: 0.8263 - val_loss: 0.1718 - val_accuracy: 0.9447
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.3633 - accuracy: 0.8886 - val_loss: 0.1166 - val_accuracy: 0.9645
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2595 - accuracy: 0.9189 - val_loss: 0.0535 - val_accuracy: 0.9847


0.9532879818594104

In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same"),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),
                    
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),
                     
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding="same"),
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu'), 
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),
                    
                    Flatten(),
                    
                    Dense(1024, activation="relu"),
                    Dropout(params['dropout_dense_block_one']),

                    Dense(1024, activation="relu"),
                    Dropout(params['dropout_dense_block_two']),

                    Dense(num_classes, activation='softmax'),
                    ])

In [0]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 6ms/step - loss: 0.2242 - accuracy: 0.9533


0.9532879590988159

In [0]:
def func_obj(params):

  model = get_model(params)

  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
            X_train, 
            y_train,
            batch_size=int(params.get('batch_size', 128)),
            epochs=5,
            verbose=0
            )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss' : -accuracy, 'status' : STATUS_OK, 'model' : model}

In [0]:
space = {
    'batch_size' : hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0.3, 0.7)

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 200.0, 'dropout_cnn_block_one': 0.34269194761577076, 'dropout_cnn_block_three': 0.43366186925043015, 'dropout_cnn_block_two': 0.4823585769959734, 'dropout_dense_block_one': 0.5905096900441498, 'dropout_dense_block_two': 0.3544239061345807}
accuracy=0.8108843564987183
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.48245813717745956, 'dropout_cnn_block_three': 0.33258569355781353, 'dropout_cnn_block_two': 0.3914101055161462, 'dropout_dense_block_one': 0.4152178810481618, 'dropout_dense_block_two': 0.3998794233451703}
accuracy=0.9054421782493591
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3718101507669438, 'dropout_cnn_block_three': 0.4566959937864456, 'dropout_cnn_block_two': 0.49025983952915353, 'dropout_dense_block_one': 0.469079789504121, 'dropout_dense_block_two': 0.4990311037779632}
accuracy=0.9276643991470337
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4834533903685571, 'dropout_cnn_block_three': 0.4764454008506492, 'dropout_cnn_block_two': 0.499384064567